In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'aok-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [3]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


C:\Users\COMPU LAND\AppData\Local\Temp\ipykernel_18764\4254666078.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [4]:
amazon_df

,title,price,rating,reviews,availability
0,Sony Playstation PS4 1TB Black Console,$298.99,4.6 out of 5 stars,"1,568 ratings",
1,"Playstation Sony 4, 500GB Slim System [CUH-221...",$224.99,4.3 out of 5 stars,374 ratings,Only 8 left in stock - order soon.
2,PlayStation 4 Slim 1TB Console - Marvel's Spid...,$271.37,4.6 out of 5 stars,"2,821 ratings",Only 1 left in stock - order soon.
3,PlayStation®4 Console – Call of Duty® Modern W...,$379.92,4.5 out of 5 stars,120 ratings,In Stock
4,PlayStation 4 Slim 1TB Console,$369.99,4.7 out of 5 stars,"15,740 ratings",
5,PlayStation 4 Slim 1TB Console,$249.99,4.2 out of 5 stars,312 ratings,
6,PlayStation 4 500GB Console - Call of Duty Bla...,$241.99,4.6 out of 5 stars,"2,005 ratings",Only 1 left in stock - order soon.
7,Sony PlayStation 4 Pro 1TB Console - Black (PS...,$249.95,4.4 out of 5 stars,"4,136 ratings",Only 1 left in stock - order soon.
8,PlayStation 4 Slim 500GB Console [Discontinued],$196.99,4.5 out of 5 stars,"1,912 ratings",Only 2 left in stock - order soon.
9,PlayStation 4 Slim 500GB Console - Uncharted 4...,$244.95,4.8 out of 5 stars,"6,500 ratings",
